## Add Custom Narrative to FHIR Search Parameter

- Get SP from file
- Generate Narrative from Jinja2 Template
- Strip out any null values
- Save SP back to file

In [9]:
from json import dumps, loads, load
from pathlib import Path
from IPython.display import display as Display, HTML, Markdown
from jinja2 import Environment, FileSystemLoader, select_autoescape
from lxml import etree
from fhir.resources import construct_fhir_element
from commonmark import commonmark
from datetime import datetime

def json_serial(obj):
    """JSON serializer for datetime objects not serializable by default json code"""

    if isinstance(obj, (datetime)):
        return obj.strftime('%Y-%m-%d')
    raise TypeError ("Type %s not serializable" % type(obj))

### Get SP from file

In [10]:
in_path = Path() / 'test' / 'in_sp.json' # temp path
sp = loads(
        in_path.read_text()
    )
sp['date'] = datetime.utcnow()  #convert to dateime object
sp

{'resourceType': 'SearchParameter',
 'id': 'us-core-patient-gender-identity',
 'url': 'http://hl7.org/fhir/us/core/SearchParameter/us-core-patient-gender-identity',
 'version': '4.1.0',
 'name': 'USCorePatientGenderIdentity',
 'status': 'active',
 'date': datetime.datetime(2023, 4, 6, 20, 40, 41, 835158),
 'publisher': 'HL7 International - Cross-Group Projects',
 'contact': [{'name': 'HL7 International - Cross-Group Projects',
   'telecom': [{'system': 'url',
     'value': 'http://www.hl7.org/Special/committees/cgp'},
    {'system': 'email', 'value': 'cgp@lists.HL7.org'}]}],
 'description': 'Returns patients with an gender-identity extension matching the specified code.',
 'jurisdiction': [{'coding': [{'system': 'urn:iso:std:iso:3166',
     'code': 'US'}]}],
 'code': 'gender-identity',
 'base': ['Patient'],
 'type': 'token',
 'expression': "Patient.extension.where(url='http://hl7.org/fhir/us/core/StructureDefinition/us-core-genderIdentity').value.coding.code",
 'xpath': "f:Patient/f:ex

### Generate Narrative from Jinja2 Template

In [11]:
def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = ''),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )


env.filters['markdown'] = markdown

template = env.get_template('sp_narrative.j2')  

rendered = template.render(sp=sp)
display(HTML(rendered))


parser = etree.XMLParser(remove_blank_text=True)
root = etree.fromstring(rendered, parser=parser)

div = (etree.tostring(root[1][0], encoding='unicode', method='html'))

my_obj = dict(
            status = 'generated',
            div = div
            )
            
narr = construct_fhir_element('Narrative', my_obj)

sp['text'] = narr.dict()

### Save SP back to file

In [12]:
out_path = Path() / 'test' / 'out_sp.json' # temp path
out_path.write_text(dumps(sp, indent = 2, default=json_serial))

3521